# pyNBL: Basketball Statistic System for Australian NBL

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ssardina/pynbl/HEAD)

This notebook **incrementally** builds a set of stat tables from NBL Basketball Games:

1. A table of games played, with team names, points, venue, etc.
2. A stat table of _stint lineups_ (advance) statistics for each game and each team. A **stint** is a lineup of players who play together in different interval periods across the game. This table will contain the stints for each team from the play-by-play data and compute various statistics for those stints.

Tables will be saved in CSV and Excel formats as well as in [Pickle format](https://docs.python.org/3/library/pickle.html) for later recovery as Panda DataFrames.


The data comes as a raw JSON file using the game id (e.g., `2087737`):

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [ ]:
# Let's first load all required packages...
import os
from pathlib import Path
import pandas as pd
import numpy as np
import dtale

from config import *
import bball_stats
import tools


# Set folder with data files and Pickle tables saved on disk
DATA_DIR='data-22_23/'
FILES = dict()
FILES['stint_stats'] = Path(DATA_DIR, "stint_stats_df").with_suffix('.pkl')
FILES['stints'] = Path(DATA_DIR, "stints_df").with_suffix('.pkl')
FILES['games'] = Path(DATA_DIR, "games_df").with_suffix('.pkl')
FILES['players'] = Path(DATA_DIR, "players_df").with_suffix('.pkl')

## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [11]:
# Games to be computed  - Format: (game id, round number)
games_21_22_john = [(1976446, 1), (1976447, 1), (1976448, 1), (1976452, 1), (1976454, 1), (2004608, 1), (2004609, 2), (1976449, 2), (1976451, 2), (1976453, 2), (1976455, 2), (1976458, 2), (1976456, 2), (1976457, 3), (1976459, 3), (1976460, 3), (1976461, 3), (1976462, 3), (1976463, 3), (1976464, 3), (2004610, 3), (1976465, 3), (1976468, 4), (1976469, 4), (1976474, 5), (1976473, 5), (1976482, 6), (2036215, 7), (2031329, 7), (2031330, 7), (2031332, 7), (2031333, 7), (2031334, 7), (2031335, 8), (2031336, 8), (2031337, 8), (2031338, 8), (2031340, 8), (2031341, 8), (2046695, 8), (2046696, 8), (2046697, 9), (2031342, 9), (2031343, 9), (2031344, 9), (2031345, 9), (2031346, 9), (2031347, 9), (2046698, 10), (2046700, 10), (2046701, 10), (2046702, 10), (2046703, 10), (2046704, 10), (2046706, 10), (2046707, 11), (2046709, 11), (2046710, 11), (2046711, 11), (2046712, 11), (2046713, 11), (2051763, 11), (2053811, 12), (2053812, 12), (2053813, 12), (2053814, 12), (2053815, 12), (2053816, 12), (2053817, 12), (2053818, 13), (2053819, 13), (2053820, 13), (2053821, 13), (2053822, 13), (2053823, 13), (2053824, 13), (2053825, 13), (2056454, 14), (2056455, 14), (2056457, 14), (2056458, 14), (2056461, 14), (2056462, 14), (2056460, 14), (2056463, 15), (2056464, 15), (2056466, 15), (2056467, 15), (2056469, 15), (2056471, 15), (2056472, 15), (2056473, 15), (2065653, 16), (2065654, 16), (2065655, 16), (2065656, 16), (2065657, 17), (2065658, 17), (2065659, 17), (2069165, 16), (2069166, 16), (2069167, 16), (2069168, 17), (2069169, 17), (2069170, 17), (2069171, 17), (2069172, 17), (2069175, 18), (2069177, 18), (2069179, 18), (2069181, 18), (2069183, 18), (2069184, 18), (2069186, 18), (2069187, 18), (2069191, 18), (2069192, 19), (2069194, 19), (2069196, 19), (2069199, 19), (2069202, 19), (2069203, 19), (2069204, 19), (2069190, 19), (2069193, 19), (2069195, 20), (2069197, 20), (2069198, 20), (2069200, 20), (2069201, 20), (2069205, 20), (2069173, 20), (2069174, 21), (2069176, 21), (2069178, 21), (2069180, 21), (2069182, 21), (2069188, 21), (2069189, 20)]
games_21_22 = [('2004608', 1), ('1976448', 1), ('1976452', 1), ('1976454', 1), ('1976446', 1), ('1976447', 1), ('1976451', 2), ('1976455', 2), ('1976456', 2), ('1976453', 2), ('1976458', 2), ('1976449', 2), ('2004609', 2), ('1976461', 3), ('1976463', 3), ('1976459', 3), ('1976460', 3), ('1976462', 3), ('1976464', 3), ('1976457', 3), ('1976465', 3), ('2004610', 3), ('1976468', 4), ('1976469', 4), ('1976474', 5), ('1976473', 5), ('1976482', 6), ('2036215', 7), ('2031330', 7), ('2031332', 7), ('2031333', 7), ('2031329', 7), ('2031334', 7), ('2046695', 8), ('2031341', 8), ('2031335', 8), ('2046696', 8), ('2031338', 8), ('2031340', 8), ('2031337', 8), ('2031336', 8), ('2046697', 9), ('2031345', 9), ('2031343', 9), ('2031347', 9), ('2031342', 9), ('2031344', 9), ('2031346', 9), ('2046702', 10), ('2046704', 10), ('2046701', 10), ('2046703', 10), ('2046698', 10), ('2046700', 10), ('2046706', 10), ('2046711', 11), ('2051763', 11), ('2046712', 11), ('2046713', 11), ('2046709', 11), ('2046710', 11), ('2046707', 11), ('2053812', 12), ('2053816', 12), ('2053811', 12), ('2053815', 12), ('2053813', 12), ('2053814', 12), ('2053817', 12), ('2053818', 13), ('2053819', 13), ('2053822', 13), ('2053823', 13), ('2053821', 13), ('2053825', 13), ('2053820', 13), ('2053824', 13), ('2056457', 14), ('2056461', 14), ('2056462', 14), ('2056460', 14), ('2056458', 14), ('2056455', 14), ('2056454', 14), ('2056467', 15), ('2056472', 15), ('2056473', 15), ('2056466', 15), ('2056464', 15), ('2056469', 15), ('2056463', 15), ('2056471', 15), ('2065657', 16), ('2065653', 16), ('2065656', 16), ('2065655', 16), ('2065654', 16), ('2065659', 16), ('2065658', 16), ('2069172', 17), ('2069165', 17), ('2069171', 17), ('2069170', 17), ('2069169', 17), ('2069166', 17), ('2069168', 17), ('2069167', 17), ('2069175', 18), ('2069184', 18), ('2069181', 18), ('2069177', 18), ('2069183', 18), ('2069179', 18), ('2069186', 18), ('2069187', 18), ('2069202', 19), ('2069203', 19), ('2069196', 19), ('2069204', 19), ('2069205', 19), ('2069191', 19), ('2069192', 19), ('2069199', 19), ('2069194', 19), ('2069195', 20), ('2069189', 20), ('2069193', 20), ('2069197', 20), ('2069201', 20), ('2069200', 20), ('2069190', 20), ('2069198', 20), ('2069176', 21), ('2069185', 21), ('2069180', 21), ('2069178', 21), ('2069174', 21), ('2069173', 21), ('2069188', 21), ('2069182', 21), ('2087734', 100), ('2087735', 100), ('2087733', 100), ('2087737', 100), ('2087736', 100), ('2090350', 101), ('2090352', 101), ('2090351', 101)]


# We build each round incrementally
games_22_23 = []
games_22_23.extend([('2122059', 0), ('2120077', 0), ('2120053', 0), ('2120079', 0), ('2120054', 0), ('2122060', 0), ('2141127', 0), ('2116579', 0), ('2135117', 0), ('2120050', 0), ('2141126', 0), ('2120049', 0), ('2135116', 0), ('2120048', 0), ('2120078', 0), ('2120056', 0), ('2134935', 0), ('2120055', 0), ('2120080', 0), ('2120058', 0), ('2116576', 0), ('2120052', 0), ('2120057', 0), ('2124207', 0), ('2120051', 0)])


DATA_DIR, games = 'data-21_22/', games_21_22
# DATA_DIR, games = 'data-22_23/', games_22_23

print("Games to scrape:", games)
print("Folder to be used:", DATA_DIR)

# Set to True to re-compute from scratch all tables
reload = True

Games to scrape: [('2004608', 1), ('1976448', 1), ('1976452', 1), ('1976454', 1), ('1976446', 1), ('1976447', 1), ('1976451', 2), ('1976455', 2), ('1976456', 2), ('1976453', 2), ('1976458', 2), ('1976449', 2), ('2004609', 2), ('1976461', 3), ('1976463', 3), ('1976459', 3), ('1976460', 3), ('1976462', 3), ('1976464', 3), ('1976457', 3), ('1976465', 3), ('2004610', 3), ('1976468', 4), ('1976469', 4), ('1976474', 5), ('1976473', 5), ('1976482', 6), ('2036215', 7), ('2031330', 7), ('2031332', 7), ('2031333', 7), ('2031329', 7), ('2031334', 7), ('2046695', 8), ('2031341', 8), ('2031335', 8), ('2046696', 8), ('2031338', 8), ('2031340', 8), ('2031337', 8), ('2031336', 8), ('2046697', 9), ('2031345', 9), ('2031343', 9), ('2031347', 9), ('2031342', 9), ('2031344', 9), ('2031346', 9), ('2046702', 10), ('2046704', 10), ('2046701', 10), ('2046703', 10), ('2046698', 10), ('2046700', 10), ('2046706', 10), ('2046711', 11), ('2051763', 11), ('2046712', 11), ('2046713', 11), ('2046709', 11), ('2046710'

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, computes stats and game info, and adds them to the initial tables of stats and games.

We start by loading all saved previous games, if any, as we want to append to that database (and we don't want to recompute them).

In [ ]:
# Load tables from saved files (if any)
saved_stint_stats_df = None
saved_stints_df = None
saved_games_df = None
saved_players_df = None
existing_games = []

if not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading recorded dataframes from files")
    try:
        saved_stint_stats_df = pd.read_pickle(FILES['stint_stats'])
        saved_stints_df = pd.read_pickle(FILES['stints'])
        saved_games_df = pd.read_pickle(FILES['games'])
        saved_players_df = pd.read_pickle(FILES['players'])
        # collect game ids of all games recovered from file
        existing_games = saved_games_df.game_id.unique()
    except FileNotFoundError as e:
        print("Error loading Pickle files: ", e)
        saved_stint_stats_df = None
        saved_stints_df = None
        saved_games_df = None
        saved_players_df = None
        existing_games = []
else:
    existing_games = []

print(f"Recovered {len(existing_games)} games: ", existing_games)

# saved_stats_df['lineup'].apply(lambda x: len(x) > 5)
# saved_stats_df.loc[5,'lineup']
# saved_stats_df.loc[5]

# saved_stint_stats_df.sample(3)
# saved_games_df.sample(3)

It is now time to process games to extract:

1. Table of **games**.
2. Table of **players** who played in each game with their stats, for each team.
3. Table of **stints** in each game for each team.
4. Table of **stint stats** in each game for each team.

In [ ]:
# collect here set of stat dfs and game info, one per game
#   then, we will put them together into different dataframes
from urllib.error import HTTPError


stint_stats_dfs = []
stints_dfs = []
players_dfs = []
games_data = []

# Build data for each game, we'll put them together after....
for game in games:
    # get game_id and round no (if available)
    if isinstance(game, tuple):
        game_id, round_no = game
    else:
        game_id = game
        round_no = np.nan # no round info available

    # don't scrape game data if already loaded from file, skip it
    if game_id in existing_games:
        print(f"Game {game_id} was already saved on file; no scrapping...")
        continue

    ##################################################################
    # !!! MAIN STEP: scrape and compute the actual stats for the game
    ##################################################################
    print(f"Computing game {game_id}...")

    # 1. Read game JSON file
    try:
        game_json = tools.get_json_data(game_id)
    except HTTPError as e:
        print(f"Game {game_id} JSON data not available yet: ", e)
        continue


    result = bball_stats.build_game_stints_stats_df(game_json, game_id)
    game_stint_stats_df = result['stint_stats_df']   #  this is basically what we care, the stint stats
    game_stints_df = result['stints_df']
    game_team1, game_team2 = result['teams']

    # Add the game id column to game tables
    game_stint_stats_df.insert(0, 'game_id', game_id)
    game_stints_df.insert(0, 'game_id', game_id)

    # Extract players in the game
    players_df = bball_stats.get_players_stats(game_json)
    players_df.insert(0, 'game_id', game_id)

    # Add tables to collected set of tables, one per game
    stint_stats_dfs.append(game_stint_stats_df)
    stints_dfs.append(game_stints_df)
    players_dfs.append(players_df)

    # Next build the record for the game dataframe
    # first, extract date of game from HTML page
    try:
        game_info = tools.get_game_info(game_id)
    except:
        game_info = { "venue" : np.nan, "date": np.nan}
    print(f"\t .... done: {game_team1[0]} ({game_team1[1]}) vs {game_team2[0]} ({game_team2[1]}) on {game_info['date']}")

    games_data.append({"game_id": game_id,
                        "date" : game_info['date'],
                        "round": round_no,
                        "team1": game_team1[0],
                        "team2": game_team2[0],
                        "s1": game_team1[1],
                        "s2": game_team2[1],
                        "winner": 1 if game_team1[1] > game_team2[1] else 2,
                        "venue" : game_info["venue"]}
                      )


#################################
# All games have been processed, now put all dfs together
#################################
if len(games_data) == 0:
    raise SystemExit("No games!")

# First, build a dataframe with all the game data collected
games_df = pd.DataFrame(games_data) if saved_games_df is None else pd.concat([saved_games_df, pd.DataFrame(games_data)])
games_df.reset_index(inplace=True, drop=True)

# Build players dataframe
players_df = pd.concat(players_dfs + ([saved_players_df] if saved_players_df is not None else []))
players_df.reset_index(inplace=True, drop=True)

# Build stint stats dataframe
stint_stats_df = pd.concat(stint_stats_dfs + ([saved_stint_stats_df] if saved_stint_stats_df is not None else []))
stint_stats_df.reset_index(inplace=True, drop=True)

# Build stints dataframe
stints_df = pd.concat(stints_dfs + ([saved_stints_df] if saved_stints_df is not None else []))
stints_df.reset_index(inplace=True, drop=True)

print("Number of total games collected: ", games_df.shape[0])
print("Number of NEW collected: ", games_df.shape[0] - len(existing_games))
print("Number of FAILED games (not yet played): ", len(games) - games_df.shape[0])

stint_stats_df.sample(2)
stints_df.sample(2)
games_df.sample(5)
players_df.sample(5)

If we want we can do some sanity checks, before saving to disk:

In [ ]:
dtale.show(games_df)
dtale.show(stint_stats_df)
# dtale.show(stints_df)
# dtale.show(players_df)

In [ ]:
import random
print("The shape of stats_df is:", stint_stats_df.shape)
stats_cols = list(stint_stats_df.columns[4:-49])
print("Stats cols:", stats_cols)

# build columns we want to show
cols = ['game_id' , 'tno', 'team', 'stint']
rnd_cols = random.sample(stats_cols, 8)
rnd_cols.extend([f"{x}_opp" for x in rnd_cols])
cols.extend(rnd_cols)

# show some sample of stats computed
stint_stats_df[cols].sample(5)

Sanity check that `(ortg, drtg)` (offensive/defensive rate goal) should mirror `(drtg_opp, ortg_opp)` (opponent offensive/defensive rate goal)):

In [ ]:
# (ortg, drtg) should mirror (drtg_opp, ortg_opp)
stint_stats_df.iloc[4][['game_id' , 'team', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [ ]:
import datetime
import os
import shutil
from pathlib import Path

# make a backup of existing tables on files
for pkl_file in FILES.values():
    for ext in ['.csv', '.xlsx', '.pkl']:
        file = Path(pkl_file).with_suffix(ext)
        if os.path.exists(file):
            # print("Backup file", file)
            shutil.copy(file, str(file) + ".bak")

# dump stint stats dataframe
stint_stats_df.to_pickle(Path(DATA_DIR, "stint_stats_df").with_suffix(".pkl"))
stint_stats_df.to_csv(Path(DATA_DIR, "stint_stats_df").with_suffix(".csv"), index=False)
stint_stats_df.to_excel(Path(DATA_DIR, "stint_stats_df").with_suffix(".xlsx"), index=False)

# dump stint stats dataframe
stints_df.to_pickle(Path(DATA_DIR, "stints_df").with_suffix(".pkl"))
stints_df.to_csv(Path(DATA_DIR, "stints_df").with_suffix(".csv"), index=False)
stints_df.to_excel(Path(DATA_DIR, "stints_df").with_suffix(".xlsx"), index=False)

# dump game dataframe
games_df.to_pickle(Path(DATA_DIR, "games_df").with_suffix(".pkl"))
games_df.to_csv(Path(DATA_DIR, "games_df").with_suffix(".csv"), index=False)
games_df.to_excel(Path(DATA_DIR, "games_df").with_suffix(".xlsx"), index=False)

# dump players dataframe
players_df.to_pickle(Path(DATA_DIR, "players_df").with_suffix(".pkl"))
players_df.to_csv(Path(DATA_DIR, "players_df").with_suffix(".csv"), index=False)
players_df.to_excel(Path(DATA_DIR, "players_df").with_suffix(".xlsx"), index=False)

now = datetime.datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

print(f"Finished saving in {DATA_DIR} @ {date_time}")

## 4. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [ ]:
dtale.show(stint_stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## 5. Some checks...

Check if a stint lineup has more than 5 players! It could happen:

1. Game 2031329, player H. Besson comes out (wrongly?) at 3rd period min 10:00 but he keeps playing and then goes out again at 7:33.

In [ ]:
stint_stats_df.shape
mask = stint_stats_df['lineup'].apply(lambda x: len(x) != 5)
stint_stats_df[mask]

# stats_df.iloc[941][['game_id', 'lineup']]